In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from keras import preprocessing, optimizers
from keras_preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%capture
!unzip /content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/images.zip -d  /content/images

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/podcast_final.csv') # all_podcast_withpaths

In [5]:
from sklearn.utils import shuffle
df = shuffle(df, random_state = 42)

In [6]:
len(df['Primary Genre'].value_counts()) #check how many target labels

35

In [7]:
filepaths = df['filepaths'].tolist() #change the paths according to colab directory
new = []
for el in filepaths:
  path = el.split('/')[-1]
  path= '/content/images/' + path
  new.append(path)
df['filepaths'] = new

In [8]:
df_train = df[:28279]
df_test = df[28279:]

In [9]:
datagen=ImageDataGenerator(rescale=1./255,validation_split=0.25)
train_generator=datagen.flow_from_dataframe(dataframe=df_train, 
                                            directory=None, x_col="filepaths", y_col="Primary Genre", 
                                            class_mode="categorical", target_size=(224,224), batch_size=32,
                                            shuffle = True, subset='training')
validation_generator=datagen.flow_from_dataframe(dataframe=df_train, 
                                            directory=None, x_col="filepaths", y_col="Primary Genre", 
                                            class_mode="categorical", target_size=(224,224), batch_size=32,
                                            shuffle = True, subset='validation')

test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
                                            dataframe=df_test,
                                            directory=None,
                                            x_col="filepaths",
                                            y_col='Primary Genre',
                                            batch_size=32,
                                            target_size=(224,224))

Found 21210 validated image filenames belonging to 35 classes.
Found 7069 validated image filenames belonging to 35 classes.
Found 1000 validated image filenames belonging to 35 classes.


In [10]:
print(validation_generator.class_indices) #returns the classed indices

{'After Shows': 0, 'Animation & Manga': 1, 'Arts': 2, 'Books': 3, 'Business': 4, 'Comedy': 5, 'Design': 6, 'Documentary': 7, 'Education': 8, 'Fashion & Beauty': 9, 'Fiction': 10, 'Food': 11, 'Games': 12, 'Health': 13, 'Hobbies': 14, 'Interviews': 15, 'Kids & Family': 16, 'Music': 17, 'Nature': 18, 'News': 19, 'Non-Profit': 20, 'Personal': 21, 'Pets & Animals': 22, 'Places & Travel': 23, 'Politics': 24, 'Religion': 25, 'Science': 26, 'Sexuality': 27, 'Society & Culture': 28, 'Sports': 29, 'Stand-Up': 30, 'TV & Film': 31, 'Technology': 32, 'True Crime': 33, 'Vehicles': 34}


In [11]:
base_model = keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape = (224, 224, 3)
)

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
# Freeze four convolution blocks
for layer in base_model.layers: #[:15]
    layer.trainable = False
# Make sure you have frozen the correct layers
for i, layer in enumerate(base_model.layers):
    print(i, layer.name, layer.trainable) #frezee the weights

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False


In [ ]:
data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal_and_vertical"), layers.RandomRotation(0.2)]
)

# Create new model on top
inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)  # Apply random data augmentation
x = tf.keras.applications.vgg16.preprocess_input(x)
# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model.output
x = keras.layers.Flatten()(x) # Flatten dimensions to for use in FC layers
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x) # Dropout layer to reduce overfitting
x = keras.layers.Dense(1024, activation = 'relu')(x)
x = keras.layers.Dropout(0.5)(x) # Dropout layer to reduce overfitting
x = keras.layers.Dense(35, activation = 'softmax')(x)
model = keras.Model(inputs=base_model.input, outputs=x, name='my_VGG16')

model.summary()

Model: "my_VGG16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0  

In [12]:
model = keras.models.load_model('/content/drive/MyDrive/Colab_Notebooks/podcast/VGG16_finetuned_1dense2048.h5') #load model to resuming training from checkpoint
model.summary()

Model: "my_VGG16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0  

In [13]:
model.compile(
    optimizer=keras.optimizers.RMSprop(lr=1e-6), #lower lr is better for finetuning
    loss="categorical_crossentropy",metrics=["accuracy", tf.keras.metrics.TopKCategoricalAccuracy(k=5)]
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [14]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

CSV_log = tf.keras.callbacks.CSVLogger('/content/drive/MyDrive/Colab_Notebooks/podcast/VGG16_log1dense2018.csv', separator=",", append=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=7,
restore_best_weights=True, monitor='val_loss')
model_checkpoint = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Colab_Notebooks/podcast/VGG16_finetuned_checkpoint.h5', monitor='val_loss', save_best_only = True, verbose = 1)

history = model.fit(x=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=100, callbacks=[early_stopping_cb, model_checkpoint, CSV_log], initial_epoch = 50)

Epoch 51/100
662/662 [==============================] - ETA: 0s - loss: 2.9234 - accuracy: 0.2110 - top_k_categorical_accuracy: 0.5122
Epoch 51: val_loss improved from inf to 3.06426, saving model to /content/drive/MyDrive/Colab_Notebooks/podcast/VGG16_finetuned_checkpoint.h5
662/662 [==============================] - 341s 494ms/step - loss: 2.9234 - accuracy: 0.2110 - top_k_categorical_accuracy: 0.5122 - val_loss: 3.0643 - val_accuracy: 0.1739 - val_top_k_categorical_accuracy: 0.4510
Epoch 52/100
662/662 [==============================] - ETA: 0s - loss: 2.9109 - accuracy: 0.2137 - top_k_categorical_accuracy: 0.5147
Epoch 52: val_loss improved from 3.06426 to 3.06209, saving model to /content/drive/MyDrive/Colab_Notebooks/podcast/VGG16_finetuned_checkpoint.h5
662/662 [==============================] - 300s 453ms/step - loss: 2.9109 - accuracy: 0.2137 - top_k_categorical_accuracy: 0.5147 - val_loss: 3.0621 - val_accuracy: 0.1759 - val_top_k_categorical_accuracy: 0.4516
Epoch 53/100
662

In [18]:
model.save('/content/drive/MyDrive/Colab_Notebooks/podcast/VGG16_finetuned_1dense2048.h5')